# Simple image cleaner.
\{Short description\}


1.   Run all the cells first (Runtime > Run all or Command/Ctrl+F9)
2.   Click on the "Upload button" to upload your images
3.   Click on "Run Script" to let the script process your images
3.   Click on "Download cleaned_images.zip" to download your images



In [10]:
#@title Upload your images (multiple selection allowed). Re-run the cell to reset the selection of images
import numpy as np
import re
import io
import os

try:
  import cv2
  # print('cv2 already installed')
except ImportError as e:
  print('installing ipywidgets')
  %pip install -q opencv-python

try:
  # print('panel already installed')
  import panel as pn
except ImportError as e:
  print('installing ipywidgets')
  %pip install -q panel
pn.extension()

try:
  # print('ipywidgets already installed')
  import ipywidgets as widgets
except ImportError as e:
  print('installing ipywidgets')
  %pip install -q ipywidgets


# from PIL import Image
from shutil import make_archive

try:
  from google.colab.patches import cv2_imshow # needed only to show images in environment
except:
  pass
# from google.colab import files  

if not os.path.exists('output_folder'): os.mkdir('output_folder')
if not os.path.exists('input_folder'): os.mkdir('input_folder')

uploader = widgets.FileUpload(accept='image/*', multiple=True, button_style='success')
# add uploaded images to folder
display(uploader)


FileUpload(value={}, accept='image/*', button_style='success', description='Upload', multiple=True)

In [11]:
# @title Process images
run_button = widgets.Button(
    description='Run script',
    disabled=False,
    button_style='success',
    tooltip='Run script',
    icon='check'
)

def process_images(b):
  print(len(uploader.metadata) > 0)
  if (len(uploader.metadata) > 0):
    for idx, metadata in enumerate(uploader.metadata): #uploader.data:
      image_name = metadata['name']
      print(image_name)
      print('Processing...')
      image_bytes = uploader.data[idx]

      nparr = np.frombuffer(image_bytes, np.uint8)
      
      # show image before
      # if 'cv2_imshow' in globals():
      #   print('Image before processing:')
      #   cv2_imshow(cv2.imdecode(nparr, cv2.IMREAD_COLOR))

      img = cv2.imdecode(nparr, cv2.IMREAD_GRAYSCALE) # cv2.IMREAD_COLOR in OpenCV 3.1

      height = img.shape[0]
      width = img.shape[1]
      img = cv2.GaussianBlur(img,(5,5),0) # blur already in the planes below

      # address brightness imbalance in the background
      # with original b/w image
      rgb_planes = cv2.split(img)
      result_norm_planes = []
      for plane in rgb_planes:
          dilated_img = cv2.dilate(plane, np.ones((14,14), np.uint8))
          bg_img = cv2.medianBlur(dilated_img, 21)
          diff_img = 255 - cv2.absdiff(plane, bg_img)
          norm_img = cv2.normalize(diff_img,None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8UC1)
          result_norm_planes.append(norm_img)
          
      result_norm = cv2.merge(result_norm_planes)

      # get just the image data
      img = cv2.threshold(result_norm,0,255,cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

      # show image after
      # if 'cv2_imshow' in globals():
      #   print('Image after processing:')
      #   cv2_imshow(img)

      # save new image
      image_file = cv2.imwrite(f"output_folder/{image_name}.png" , img)
    
    make_archive('cleaned_images', 'zip', 'output_folder')
    print("Finished!")
    display(pn.widgets.FileDownload(file='cleaned_images.zip', embed=True))
  else:
    print('Please upload some images first')

run_button.on_click(process_images)
run_button

Button(button_style='success', description='Run script', icon='check', style=ButtonStyle(), tooltip='Run scrip…

True
Leon P. 1937.8.20. 70835. ALS. image. p3.jpg
Processing...
Leon P. 1937.8.20. 70835. ALS. image. p2.jpg
Processing...
Leon P. 1937.8.20. 70835. ALS. image. p1.jpg
Processing...
Finished!


FileDownload(_transfers=1, data='data:application/zip;base..., embed=True, file='cleaned_images.zip', filename='cleaned_images.zip', label='Download cleaned_images.z...)